# Off-diagonal filter test
This file demonstrates the off-diagonal element filter. This filter 
does not need to be recompiled to sweep and is suited to use with JAX.

In [1]:
# This file implements and tests recursion relations
import numpy as np
from matplotlib import pyplot as plt

from jax import jit

# for importing parsed codes
import sys
import cProfile
sys.path.insert(1,'../python_source/MHD_parsed/')
sys.path.insert(1,'../python_source/')
from chiphifunc import *
from math_utilities import *
from chiphiepsfunc import *
from chiphifunc_test_suite import *
from equilibrium import *

from leading_orders import *


import MHD_parsed
debug_path = '../test_data_eduardo/'

## Initializing an `Equilibrium`
### Required quantities and orders
Unknowns:

1. X_coef_cp: 2
2. Y_coef_cp: 2
3. Z_coef_cp: 2
4. B_psi_coef_cp: 0
5. B_theta_coef_cp: 2
6. iota_coef: 0
7. Delta_coef_cp: 3
8. p_perp_coef_cp: 3

Known constants:
1. B_denom_coef_c: 2
2. B_alpha_coef: 1
3. kap_p
4. dl_p 
5. tau_p
6. eta

In [2]:
%%time
equilibrium_init = circular_axis()

filter test [ 5.90527705e-19+9.06136113e-17j  5.16706204e-18-4.78372522e-02j
 -5.38786976e-18-9.81928489e-02j -8.45769960e-19+7.10065066e-03j
 -5.87468763e-19+4.59359652e-03j -1.61834301e-18-5.76486688e-04j
 -1.42791693e-18-2.03209766e-04j -7.80974177e-19+3.86822747e-05j
  1.26404965e-17+8.35793339e-06j -1.60262718e-17-2.34009673e-06j
 -5.74269285e-18-3.00028526e-07j -5.39777459e-18+1.39416354e-07j
 -6.11564672e-19+1.49189985e-08j -8.35133911e-19-5.29459143e-09j
  6.11190772e-18+4.44769879e-10j  1.29292081e-17+3.18143969e-09j
 -7.91185015e-17-5.10645180e-10j -1.71141138e-17+2.16915705e-10j
 -4.39161176e-18-9.21251728e-10j  7.89757659e-18-2.56176940e-10j
 -5.60839861e-19-9.32614366e-09j  7.34640988e-19-6.53810797e-09j
  6.17367501e-18-5.09753595e-09j  8.33362020e-18-9.30424731e-09j
 -1.20408377e-16-5.90313946e-09j  7.75847602e-18-6.07490216e-09j
  2.35881204e-17-2.77695429e-09j -9.02445721e-18+5.10376465e-09j
 -8.58818880e-19+3.17905017e-09j  6.36035961e-18+8.40951982e-09j
 -7.75165628e

# Iterate 2 orders using the full MHD equations

In [3]:
equilibrium_init

In [4]:
iterate_2_jit = jit(iterate_2, static_argnums=(5, 6,))

In [5]:
%%time
equilibrium_a = iterate_2_jit(
    equilibrium_init,
    n_eval=4,
    B_alpha_nb2=0,
    B_denom_nm1=ChiPhiFunc(jnp.array([
        [0.   +0.j   ],
        [0.005+0.005j],
        [0.005-0.005j],
        [0.   +0.j   ]]), 1), 
    B_denom_n=ChiPhiFuncSpecial(0),
    iota_new=-0.66367278,
    max_freq=(100, 100),
    max_k_diff_post_inv=(100, 100)
)

TypeError: iterate_2() got an unexpected keyword argument 'max_freq'

In [6]:
%%time
equilibrium_b = iterate_2_jit(
    equilibrium_init,
    n_eval=4,
    B_alpha_nb2=0,
    B_denom_nm1=ChiPhiFunc(jnp.array([
        [0.   +0.j   ],
        [0.005+0.005j],
        [0.005-0.005j],
        [0.   +0.j   ]]), 1), 
    B_denom_n=ChiPhiFuncSpecial(0),
    iota_new=-0.66367278,
    max_freq=(100, 100),
    max_k_diff_post_inv=(50, 100)
)

TypeError: iterate_2() got an unexpected keyword argument 'max_freq'

In [7]:
%%time
equilibrium_c = iterate_2_jit(
    equilibrium_init,
    n_eval=4,
    B_alpha_nb2=0,
    B_denom_nm1=ChiPhiFunc(jnp.array([
        [0.   +0.j   ],
        [0.005+0.005j],
        [0.005-0.005j],
        [0.   +0.j   ]]), 1), 
    B_denom_n=ChiPhiFuncSpecial(0),
    iota_new=-0.66367278,
    max_freq=(100, 100),
    max_k_diff_post_inv=(20, 100)
)

TypeError: iterate_2() got an unexpected keyword argument 'max_freq'

## Accessing data and checking accuracy

In [8]:
(J, Cb, Ck, Ct, I, II, III) = equilibrium_a.check_governing_equations(4)
J.display_content()
Cb.display_content()
Ck.display_content()
Ct.display_content()
I.display_content()
II.display_content()
III.display_content()


NameError: name 'equilibrium_a' is not defined

In [ ]:
(J, Cb, Ck, Ct, I, II, III) = equilibrium_b.check_governing_equations(4)
J.display_content()
Cb.display_content()
Ck.display_content()
Ct.display_content()
I.display_content()
II.display_content()
III.display_content()


In [ ]:
(J, Cb, Ck, Ct, I, II, III) = equilibrium_c.check_governing_equations(4)
J.display_content()
Cb.display_content()
Ck.display_content()
Ct.display_content()
I.display_content()
II.display_content()
III.display_content()


In [ ]:
a = np.random.random((100,100))
b = np.random.random((1000,1000))
c = np.identity(1000)
c[:100, :100] = a

In [ ]:
%%time
jnp.linalg.inv(a)
print()

In [ ]:
%%time
jnp.linalg.inv(b)
print()

In [ ]:
%%time
jnp.linalg.inv(c)
print()

In [ ]:
jnp.linalg.inv(a)